In [283]:
import pandas as pd

In [284]:
df = pd.read_csv('./data/gemstone.csv')
df.head()
df=df.drop(labels=['id'],axis=1)
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [285]:
df.tail()

,carat,cut,color,clarity,depth,table,x,y,z,price
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681
193572,0.71,Good,E,SI2,60.8,64.0,5.73,5.71,3.48,2258


In [286]:
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['number']).columns
print(numerical_columns)
print(categorical_columns)

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')
Index(['cut', 'color', 'clarity'], dtype='object')


In [287]:
import seaborn as sns
import matplotlib.pyplot as plt
# cut_map={'Fair':1,'Good':2,'Very Good':3,'Premium':4,'Ideal':5}
# clarity_map={'VS2':4, 'SI2':2, 'VS1':5, 'SI1':3, 'IF':8, 'VVS2':6, 'VVS1':7, 'I1':1}
# color_map={'F':3, 'J':7, 'G':4, 'E':2, 'D':1, 'H':5, 'I':6}
# df['cut']=df['cut'].map(cut_map)
# df['clarity']=df['clarity'].map(clarity_map)
# df['color']=df['color'].map(color_map)

In [288]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [289]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
#pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [290]:
#Numerical_Pipeline

# Create a transformer for numerical features
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with mean
    ('scaler', StandardScaler())  # Standardize features by removing the mean and scaling to unit variance
])

#Categorical PIPELINE
# Create a transformer for categorical features
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent value
    ('ordinalencoder', OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),  # One-hot encode categorical features
    
    ('scaler', StandardScaler()) 
])

# Create a ColumnTransformer to apply different preprocessing steps to numerical and categorical columns
preprocessor = ColumnTransformer([
    ('numerical_pipeline', numerical_pipeline, numerical_columns),  # Apply numerical pipeline to numerical columns
    ('categorical_pipeline', categorical_pipeline, categorical_columns)  # Apply categorical pipeline to categorical columns
])

In [291]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)


In [292]:
X_train.head()

,carat,cut,color,clarity,depth,table,x,y,z
168192,0.34,Ideal,I,VVS2,60.9,57.0,4.56,4.53,2.76
35202,0.90,Good,E,SI1,63.8,57.0,6.07,6.03,3.87
41091,1.02,Premium,G,VS1,62.7,58.0,6.35,6.39,4.00
31239,0.32,Premium,G,VS2,62.1,59.0,4.37,4.35,2.71
45722,0.35,Ideal,J,VVS2,61.1,56.0,4.53,4.57,2.78


In [293]:
# Assuming you have preprocessed features stored in X_train_processed
X_train_processed = preprocessor.fit_transform(X_train)

# Get column names after transformation
numerical_feature_names = numerical_columns.tolist()  # assuming numerical_columns is a variable storing numerical column names
categorical_feature_names = categorical_columns.tolist()  # assuming categorical_columns is a variable storing categorical column names
feature_names = numerical_feature_names + categorical_feature_names

# Create a new DataFrame with processed features
X_train = pd.DataFrame(X_train_processed, columns=feature_names)

# Now X_train contains the preprocessed features in DataFrame format
# Assuming you have preprocessed features stored in X_test_processed
X_test_processed = preprocessor.transform(X_test)  # Use transform, not fit_transform for test data

# Create a new DataFrame with processed features for X_test
X_test= pd.DataFrame(X_test_processed, columns=feature_names)

# Now X_test_contains the preprocessed features in DataFrame format



In [294]:
X_test

,carat,depth,table,x,y,z,cut,color,clarity
0,-0.564688,-0.942132,-0.642862,-0.429765,-0.464061,-0.500036,-0.132136,-0.935071,0.019720
1,-0.175556,1.000906,-0.121531,-0.042137,-0.028595,0.036132,-1.138347,0.912774,-0.646786
2,-1.061913,0.260701,-0.121531,-1.304180,-1.298703,-1.268060,0.874076,0.912774,2.685743
3,0.970223,-0.201927,1.963794,1.048629,0.996563,0.978049,-0.132136,0.296826,0.019720
4,-0.932202,-1.312235,0.399800,-1.006699,-0.990248,-1.065186,-0.132136,-0.935071,0.686225
...,...,...,...,...,...,...,...,...,...
58067,1.013460,1.185958,-0.642862,1.003556,1.041924,1.151941,-1.138347,0.912774,0.019720
58068,-0.997058,0.260701,-1.164193,-1.141917,-1.126331,-1.108659,0.874076,-0.319122,2.019237
58069,-0.197174,-3.347799,1.442462,0.102096,0.071199,-0.224706,-0.132136,2.144670,0.019720
58070,-0.824110,-0.201927,-0.121531,-0.853450,-0.881382,-0.876803,0.874076,0.296826,-0.646786


In [301]:
# Train multipe models
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
import numpy as np

# Assuming you have preprocessed features stored in X_train_processed and X_test_processed, and targets in y_train and y_test
best_r2_score = -1  
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Lasso Regression': Lasso(),
    'Ridge Regression': Ridge(),
    'ElasticNet': ElasticNet()
}

# Train and evaluate models
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_predictions = model.predict(X_test)
    
    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, y_predictions)
    mae = mean_absolute_error(y_test, y_predictions)
    rmse = sqrt(mse)
    r2 = r2_score(y_test, y_predictions)
    
    # Print metrics
    print(f"Metrics for {name}:")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R-squared (R2): {r2}")
    print("="*30)
    
    # if rmse < best_rmse:
    #     best_rmse = rmse
    #     best_model_name = name
    if r2 > best_r2_score:
        best_r2_score = r2
        best_model_name = name

# Notify the best model
print(f"\nThe best model is '{best_model_name}' with an R-squared (R2) score of {best_r2_score:.2f}")


Metrics for Linear Regression:
Mean Squared Error (MSE): 1028002.7598132554
Mean Absolute Error (MAE): 674.0255115796858
Root Mean Squared Error (RMSE): 1013.9047094344002
R-squared (R2): 0.9368908248567511
Metrics for Lasso Regression:
Mean Squared Error (MSE): 1027949.4559693959
Mean Absolute Error (MAE): 675.071692336216
Root Mean Squared Error (RMSE): 1013.8784226767013
R-squared (R2): 0.9368940971841704
Metrics for Ridge Regression:
Mean Squared Error (MSE): 1028005.22936776
Mean Absolute Error (MAE): 674.0555800798152
Root Mean Squared Error (RMSE): 1013.9059272771611
R-squared (R2): 0.9368906732505942
Metrics for ElasticNet:
Mean Squared Error (MSE): 2351365.3822896425
Mean Absolute Error (MAE): 1060.7368759154729
Root Mean Squared Error (RMSE): 1533.416245606405
R-squared (R2): 0.8556494831165181

The best model is 'Lasso Regression' with an R-squared (R2) score of 0.94


In [298]:
model

ElasticNet()